### Setup

In [ ]:
import logging
from multiprocessing import cpu_count

import matplotlib.pyplot as plt
import numpy as np

from bounded_rand_walkers.data_generation import multi_random_walker
from bounded_rand_walkers.functions import Tophat_1D, Tophat_2D

logging.basicConfig(level=logging.INFO)

### 1D Case

In [ ]:
step_values, positions = multi_random_walker(
    n_processes=cpu_count(),
    f_i=Tophat_1D(width=0.5, centre=0.2).pdf,
    bounds=np.array([0, 1]),
    steps=int(1e4),
)

print("data shapes")
print((step_values.shape, positions.shape))

fig, axes = plt.subplots(1, 2, squeeze=True)
axes[0].hist(step_values, bins="auto")
axes[0].set_title("Step Values")
axes[1].hist(positions, bins="auto")
axes[1].set_title("Positions")
plt.show()

### 2D Case

In [ ]:
# bounds = np.array([
#     [0, 0],
#     [0, 1],
#     [1, 1],
#     [1, 0]],
#     dtype=np.float64
#     )
bounds = np.array([[1, 0], [-1, 1], [-1, -1]])

step_values, positions = multi_random_walker(
    n_processes=4,
    f_i=Tophat_2D(extent=1.5).pdf,
    bounds=bounds,
    steps=int(1e5),
    blocks=3,
)

fig, axes = plt.subplots(1, 2, squeeze=True)
fig.subplots_adjust(right=0.8)
# bin first time to get the maximum bin counts,
# which are used below
steps_bin = axes[0].hexbin(*step_values.T)
positions_bin = axes[1].hexbin(*positions.T)
axes[0].cla()
axes[1].cla()

# use this max value with the hexbin vmax option
# in order to have the same colour scaling for both
# hexbin plots, such that the same colorbar may be used
max_value = np.max([np.max(steps_bin.get_array()), np.max(positions_bin.get_array())])
steps_bin = axes[0].hexbin(*step_values.T, vmin=0, vmax=max_value)
steps_bin = axes[0].hexbin(*step_values.T, vmin=0, vmax=max_value)
positions_bin = axes[1].hexbin(*positions.T, vmin=0, vmax=max_value)
positions_bin = axes[1].hexbin(*positions.T, vmin=0, vmax=max_value)

axes[0].set_title("Step Values")
axes[1].set_title("Positions")
for ax in axes:
    ax.set_aspect("equal")

cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(positions_bin, cax=cbar_ax)

plt.show()